# Corrotina

Pode receber valores do chamador e gerar valores.

`yield` normalmente aparece do lado direito de uma expressão -> `expression = yield`.

Senão houver nada após a expressão `yield`, o gerador retorna `None`


Valores podem ser enviados pelo `.send(...)`, que então tornam-se os valores do `yield`

A PEP 380 fez alterações para que funções geradoras pudessem devolver algo com `return` (antes levantava `SyntaxError`). A sintaxe `yield from` permitiu refatorar as `funções geradoras` em funções menores.

In [1]:
def simple_coroutine():
    print(' -> coroutine started')
    x = yield
    print(' -> coroutine received:', x) # Printa o valor e termina a corrotina enviando StopIteration


In [2]:
c = simple_coroutine()
c

<generator object simple_coroutine at 0x7f1fc4d9df20>

### Preparando a função
É preciso avançar a função geradora até o primeiro yield, senão irá gerar o erro abaixo

In [3]:
c.send(10)

TypeError: can't send non-None value to a just-started generator

In [4]:
next(c)
c.send(10)

 -> coroutine started
 -> coroutine received: 10


StopIteration: 

In [5]:
def coro2(a):
    print('Iníciou com ', a)
    b = yield
    print('Recebeu: ', b)
    c =  yield a + b # Recebe novo valor e retornar soma de a + b
    print('D recebeu: ', c)
    
c2 = coro2(10)
next(c2)

Iníciou com  10


In [6]:
c2.send(10)

Recebeu:  10


20

In [7]:
c2.send(10)

D recebeu:  10


StopIteration: 

## Calculando a média com corrotina

In [8]:
def average():
    total = 0
    count = 0
    average = 0
    while True:
        received = yield average # Recebe o valor e retorna a média
        count += 1
        total += received
        average = total / count

In [9]:
a = average()
next(a)

0

In [10]:
a.send(10)

10.0

In [11]:
a.send(50)

30.0

## Preparando a função de forma pythonica

Podemos usar um decorador para lidar com essa questão de avançar até o primeiro `yield`

In [12]:
from functools import wraps


def coroutine(func):
    @wraps(func)
    def decorated(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return decorated

In [13]:
@coroutine
def averager():
    total = 0
    count = 0
    average = 0
    while True:
        received = yield average # Recebe o valor e retorna a média
        count += 1
        total += received
        average = total / count

In [14]:
media = averager()
media.send(10)

10.0

In [15]:
media.send(50)

30.0

In [16]:
media.send(60)

40.0

## Métodos adicionais

- `.close()` -> termina a corrotina
- `.throw()` -> faz a corrotina lançar uma excessão


# Retornando valores pela corrotina

In [17]:
from collections import namedtuple


Result = namedtuple("Result", "average total count")


@coroutine
def return_average():
    total = 0
    count = 0
    average = 0
    while True:
        received = yield average # Recebe o valor e retorna a média
        if not received:
            break
        count += 1
        total += received
        average = total / count
    return Result(average, total, count) # retorna o valor, e sobe a excessão StopIteration

In [18]:
av = return_average()
av.send(10)

10.0

In [19]:
av.send(50)

30.0

In [20]:
av.send(None)

StopIteration: Result(average=30.0, total=60, count=2)

In [21]:
av = return_average()
av.send(10)
av.send(50)
try:
    av.send(None)
except StopIteration as ex: # hackeando a corrotina e pegando o valor de retorno
    print(ex.value)

Result(average=30.0, total=60, count=2)


# Usando yield from

In [22]:
def gen():
    for it in 'AB':
        yield it
    for i in range(2):
        yield i
        
list(gen())

['A', 'B', 0, 1]

In [23]:
def gen():
    yield from 'AB'
    yield from range(2)
        
list(gen())

['A', 'B', 0, 1]

In [24]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [25]:
list(chain('ABC', (1, 2, 3)))

['A', 'B', 'C', 1, 2, 3]